# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [0]:
# imports
import os
import requests
import json 
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama


In [0]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
OLLAMA_API = "http://localhost:11434/api/chat"


In [0]:
# Prompts

system_prompt = "You are a tutor and helps with the user questions in detail with markdown respond with key point \
considering the recent development around the world, keep the response in most appropriate tone \n"

system_prompt += "Some of Examples are"
system_prompt += """
{"question": "1+1?", "response": "2"},
{"question": "why we shouls learn LLM Models?", "response": " Learning about Large Language Models (LLMs) is important because they are a rapidly evolving technology with the potential to significantly impact various industries, offering advanced capabilities in text generation, translation, information retrieval, and more, which can be valuable for professionals across diverse fields, allowing them to enhance their work and gain a competitive edge by understanding and utilizing these powerful language processing tools.\ 
Key reasons to learn about LLMs:\
Career advancement:\
Familiarity with LLMs can open up new career opportunities in fields like AI development, natural language processing (NLP), content creation, research, and customer service, where LLM applications are increasingly being implemented. \
Increased productivity:\
LLMs can automate repetitive tasks like writing emails, summarizing documents, generating reports, and translating text, freeing up time for more strategic work. \
Enhanced decision-making:\
By providing insights from large datasets, LLMs can assist in informed decision-making across various industries, including business, healthcare, and finance. \
Creative potential:\
LLMs can be used to generate creative content like poems, stories, scripts, and marketing copy, fostering innovation and new ideas. \
Understanding the technology landscape:\
As LLMs become increasingly prevalent, understanding their capabilities and limitations is crucial for navigating the evolving technological landscape. \
What is a large language model (LLM)? - Cloudflare\
A large language model (LLM) is a type of artificial intelligence (AI) program that can recognize and generate text, among other t...\
 "},
{"question": "what is the future of AI?", "response": "AI is predicted to grow increasingly pervasive as technology develops, revolutionising sectors including healthcare, banking, and transportation"},
"""


In [0]:
# set up environment
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [0]:
# here is the question; type over this to ask something new

user_question = """
How important it is for a Data Engineers to learn LLM, Considering the evolution of AI now a days?.
"""

In [0]:
# Get gpt-4o-mini to answer, with streaming
def ask_tutor(question):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": question},
            {"role": "user", "content": system_prompt}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

# call the gpt-4o-mini to answer with streaming
ask_tutor(user_question)

In [0]:
# Get Llama 3.2 to answer
messages = [
    {"role": "user", "content": user_question}
]
HEADERS = {"Content-Type": "application/json"}
payload = {
        "model": MODEL_LLAMA,
        "messages": messages,
        "stream": True
    }

response = ollama.chat(model=MODEL_LLAMA, messages=messages)
reply = response['message']['content']
display(Markdown(reply))

# # Process the response stream
# for line in response.iter_lines():
#     if line:  # Skip empty lines
#         try:
#             # Decode the JSON object from each line
#             response_data = json.loads(line)
#             if "message" in response_data and "content" in response_data["message"]:
#                 print(response_data["message"]["content"])
#         except json.JSONDecodeError as e:
#             print(f"Failed to decode JSON: {e}")
